# 하이퍼 매개 변수 튜닝

아래 파이썬 코드는 UC Irvine Machine Learning repository에 저장되어 있는 auto-mpg 데이터를 불러와 Estimator를 사용하여 XG Boost 모델을 실행하고 Azure Machine Learning Service의 기능인 hyperparameter tuning 방법을 활용해 나온 최적의 모델을 비교합니다. 

## Azure ML 설정 (Configuration)

아래 코드는 ML workspace 연동과 computing resource 지정 등을 수행합니다.

In [ ]:
import sys
import warnings
import azureml.core
import os
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import AmlCompute, ComputeTarget

warnings.simplefilter("ignore")

subscription_id = '94ff7c1e-50c0-4466-a33b-232a0ccff39d'
resource_group = 'amlStudy1'
workspace_name = 'amlMPG'
workspace_region = 'eastus'

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

ws.write_config()

experiment_name = 'sklearn-mpg'
exp = Experiment(workspace = ws, name = experiment_name)

# GPU VM
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "mpgCPU")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target! Just use ' + compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size, 
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

print('Computing resources attached!')

## 스크립트 폴더 지정

이번 예제에서는 XG Boost 모델이 저장되어 있는 스크립트 파일을 불러와 학습을 하게 됩니다. 아래 코드는 현재 directory에 Training이라는 폴더를 만드는 작업을 수행합니다.

In [ ]:
import os
script_folder = os.path.join(os.getcwd(), "Training")
os.makedirs(script_folder, exist_ok=True)
script_folder

## Python 스크립트 파일 만들기

아래 코드는 XG Boost 모델을 파이썬 스크립트로 생성하는 작업을 수행합니다. 앞선 예제 '1.Model_ML'에서 사용되었던 코드가 대부분 사용되었고 XG Boost 모델의 hyperparameter 값만 argument parser를 이용하여 스크립트 외부에서 불러오는 방식입니다. 

In [ ]:
%%writefile $script_folder/train_xgb.py

import pandas as pd
import numpy as np
import argparse
import os
import xgboost as xgb

from azureml.core import Run

def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true))

dataset_root = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

data = pd.read_csv(dataset_root, delim_whitespace = True, header=None,
                   names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model', 'origin', 'car_name'])

data=data[data.horsepower != '?']
data.horsepower = data.horsepower.astype(float) 

from sklearn.model_selection import train_test_split

x_name = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration']
y_name = ['mpg']

x = data[x_name]
y = data[y_name]

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=123)

parser = argparse.ArgumentParser()
parser.add_argument('--objective', type=str, dest='obj')
parser.add_argument('--colsample_bytree', type=float, dest='colsamp')
parser.add_argument('--learning_rate', type=float, dest='l_rate')
parser.add_argument('--max_depth', type=int, dest='m_dep')
parser.add_argument('--alpha', type=int, dest='alp')
parser.add_argument('--n_estimators', type=int, dest='n_est')
args = parser.parse_args()

xb = xgb.XGBRegressor(objective=args.obj , 
                      colsample_bytree=args.colsamp , 
                      learning_rate=args.l_rate , 
                      max_depth=args.m_dep , 
                      alpha=args.alp , 
                      n_estimators=args.n_est , 
                      seed=123).fit(train_x, train_y)
xb_y = xb.predict(test_x)

act_y = list(test_y['mpg'])
mape_comp = []
mape_comp.append(mape(act_y, xb_y))

res = mape(act_y, xb_y)

run = Run.get_context()
run.log('mape', res)

## Estimator 사용하기

Azure Machine Learning Service SDK에 포함되어 있는 Estimator 클래스를 이용한다면 머신러닝 모델링 작업이 편리해지고 정형화됩니다. Estimator는 정해진 directory에 있는 파이썬 파일을 불러와 지정한 compute resource를 사용하여 스크립트를 실행하게 됩니다. 만약 Estimator 사용 시 conda package와 pip package를 잘 설정하지 않을 경우 스크립트를 실행할 때 에러가 발생할 수 있습니다.

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--objective': 'reg:linear' ,
    '--colsample_bytree': 0.6 ,
    '--learning_rate': 0.01 ,
    '--max_depth': 5 ,
    '--alpha': 10 ,
    '--n_estimators': 800
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train_xgb.py',
                conda_packages=['scikit-learn'] ,
                pip_packages=['pandas','numpy','xgboost'])

Estimator 설정을 완료한 후 submit을 사용하여 ML Service에서 실험을 실행할 경우 Estimator가 실행됩니다. Submit을 사용하여 첫 실행을 할 경우 완료까지 약 10분이 소요됩니다. Experiment가 실행될 때 다음과 같은 프로세스가 발생합니다.

   1. Image creation: Estimator에 저장되어 있는 요소들로 Docker 이미지가 생성됩니다. 이 이미지는 ML Service workspace의 Azure Container Registry에 저장이 되며 약 5분이 소요됩니다.
   2. Scaling: 만약 실험을 실행하기 위해 원격 클러스터에서 지정해 놓은 노드 수보다 더 많은 수가 필요하다면 Scaling 작업이 수행되고 약 5분 정도 소요됩니다.
   3. Running: 실헙을 실제로 수행하기 위해 entry_script와 파일을 compute_target으로 보내게 됩니다.
   4. Post-processing: 실행한 실험결과가 workspace에 저장됩니다.

In [ ]:
run = exp.submit(est)
run

Azure ML의 기능인 widget 클래스의 RunDetails를 사용한다면 현재 실행되고 있는 실험의 경과를 확인할 수 있습니다. 이 경과는 Azure portal에서도 확인할 수 있습니다.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

모든 run이 종료되고 get_metrics를 사용하면 지정해놓은 metric 값을 불러오게 됩니다. 이번 예제의 metric으로는 '1.Model_ML'에서 역시 사용되었던 MAPE(Mean Absolute Percentage Error)가 다시 사용되었습니다. 그리고 첫 번째 예제에서 나온 XG Boost MAPE 값과 동일한 0.115982가 나왔습니다.

In [ ]:
print(run.get_metrics())

## Hyperdrive 사용하기

머신러닝 모델링 작업을 Estimator를 사용하여 만들었다면 Azure ML SDK의 Hyperdrive 클래스를 활용하여 hyperparameter tuning이 가능합니다. Hyperparameter는 discrete일 수도 있고 continuous일 수도 있습니다. Discrete인 경우 choice나 quniform 등을 사용하면 되고 continous인 경우 uniform이나 normal 등을 사용하면 됩니다. 또한, hyperdrive 사용 시 hyperparameter sampling 방법으로는 random parameter sampling, grid parameter sampling, bayesian parameter sampling 등이 있습니다. 자세한 내용은 아래 문서를 확인하세요.

Azure Docs: https://docs.microsoft.com/ko-kr/azure/machine-learning/service/how-to-tune-hyperparameters

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveRunConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, uniform, normal

ps = RandomParameterSampling(
    {      
        '--colsample_bytree': uniform(0.1, 0.9) ,
        '--learning_rate': choice(0.001, 0.005, 0.01, 0.05, 0.1, 0.5) ,
        '--max_depth': choice(1, 2, 3, 4, 5) ,
        '--alpha': choice(1, 2, 5, 10, 20, 30) ,
        '--n_estimators': choice(100, 200, 400, 800, 1000, 5000)
    }
)

Hyperdrive의 설정 후 다시 Estimator를 설정을 합니다. 이전 Estimator와 다른 점은 script_params에서 여러 parameter 값을 지정했던 것과 다르게 objective만 regression으로 넘겨줍니다.

In [ ]:
est2 = Estimator(source_directory=script_folder,
                 script_params={'--objective': 'reg:linear'},
                 compute_target=compute_target,
                 entry_script='train_xgb.py',
                 conda_packages=['scikit-learn'] ,
                 pip_packages=['pandas','numpy','xgboost'])

BanditPolicy는 성능이 떨어지는 run을 자동으로 종료하는 역할을 합니다. 이를 통해 불필요한 리소스 사용을 최소화할 수 있습니다. 아래 코드에서는 Slack factor라는 정책이 사용되었습니다.

In [ ]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

Hyperdrive에 필요한 설정 값을 모두 지정해주었다면 HyperDriveRunConfig를 통해 최종적으로 모델링을 진행하기 위해 필요한 취합하게 됩니다. 이번 예제의 경우 20번의 다른 hyperparameter 값을 이용한 run을 총 4개의 노드에서 동시에 실행하게 됩니다. 모델의 성능을 평가하는 지표로는 MAPE가 사용되며 이 MAPE가 최소화 되는 모델을 찾는 것으로 설정하였습니다.

In [ ]:
hdc = HyperDriveRunConfig(estimator=est2 , 
                          hyperparameter_sampling=ps , 
                          policy=policy , 
                          primary_metric_name='mape' , 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE , 
                          max_total_runs=20 ,
                          max_concurrent_runs=4)

앞서 Estimator를 사용하여 실험을 run했던 것과 동일한 방법으로 실험을 run 합니다.

In [ ]:
hdr = exp.submit(hdc)

RunDetails를 사용하여 실행되고 있는 실험을 모니터링 할 수 있습니다.

In [ ]:
RunDetails(hdr).show()

약 10분 정도의 시간이 지난 후 가장 성능이 좋았던 모델의 hyperparameter 값과 MAPE 값을 불러옵니다. 아래와 같은 hyperparameter 값을 설정할 결과 tuning하지 않은 값(0.115982)보다 소폭 감소한 것을 확인할 수 있습니다.

In [ ]:
best_run = hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_metrics())

이처럼 여러 hyperparameter 값을 다양한 parameter sampling을 잘 사용한다면 지정한 metric에 해당하는 최적의 모델을 찾을 수 있습니다. 하지만 그 만큼 시간이 오래 소요된다면 단점이 있어 어느 정도의 trade-off 성격을 잘 이해한다면 유용하게 사용될 수 있습니다.